<a href="https://colab.research.google.com/github/annikaaross/Homochirality-project/blob/lio/History_reader_v001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
from google.colab import widgets
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

Ok, so what am I doing?

I'm trying to put together a notebook for looking at histories that have come out of automated (or other previous) runs. That is basically just the plotting dashboard but with imported csvs instead of producing them. Let's try it.

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Ok, so we have Drive mounted. Let's get some data.

In [12]:
# Set the working directory
# wdir = '/content/drive/Shared drives/Homochirality/output/'
# # And the batch file
# batch = '1594736419.299745/'
# # And the individual file
# run = '1594736426.434924.csv'

# Get the file we want

path1 = '/content/drive/Shared drives/Homochirality/output/*148x144.100.100.40.028.100.s.000.100.1594827865.csv'
path2 = '/content/drive/Shared drives/Homochirality/output/100x100.050.050.40.050.050.s.050.050.1594830149.csv'

path = path2



# Now we can access that file like this:
stats = pd.read_csv(path)

# Fantastic.

In [13]:
# Ok, we need to get the parameters from these filenames.

 

chunks = path.split("/")[-1].split(".")
if chunks[0][0] == "*":
  chunks[0] = chunks[0][1:]
pool, itns = tuple([int(c) for c in chunks[0].split("x")])
bbbp,lf,n,pc,bp = tuple([int(c) for c in chunks[1:6]]) 
print(f"pool = {pool}\nitns = {itns}\nbbbp = {bbbp}\nlf = {lf}\nn = {n}\npc = {pc}\nbp = {bp}")



pool = 100
itns = 100
bbbp = 50
lf = 50
n = 40
pc = 50
bp = 50


That's still a bit clunky, because we have to get those absurd filenames by hand, but hey, it's a start. Plus we have a shared drive option which makes it so file paths aren't going to be a huge issue for everyone who uses the project.

Now I want to try to do some curve fitting? But not with plotly, because I want to be able to extract info from these curves. Going with scipy.

In [14]:
# Import curve fitting package from scipy
from scipy.optimize import curve_fit

df = stats[stats["Type"]=="Polymer"]

def asymptotic(x,a,b):
  return a + b/x

def horizontal(x,b):
  return b

pars, cov = curve_fit(f=asymptotic, xdata=list(df["Length"]), ydata=list(df['%Homochirality']), p0=[0,0], bounds=(-np.inf, np.inf))

And now let's make some graphs. :)

### Plotting an individual run

In [31]:


plots = ["polyspread plotly","leftright chirality",
         "homochirality vs length","Homochirality vs age", 
         "homochirality over time"]
tb = widgets.TabBar(plots)



with tb.output_to("polyspread plotly"):
  df = stats
  counts = df.groupby("Iteration")['Signed ee'].value_counts().reset_index(name='count')
  fig = px.scatter(counts, x="Iteration", y="Signed ee", size='count')
  fig.show()


with tb.output_to("leftright chirality"):

  df = stats
  lefts = df.groupby("Iteration")["#LeftHomochiral"].sum().rename("LL")
  rights = df.groupby("Iteration")["#RightHomochiral"].sum().rename("RR")
  heteros = (df.groupby("Iteration")['Length'].sum() - df.groupby("Iteration")['Length'].count()).rename("LR")
  bondcounts = pd.DataFrame([lefts, rights, heteros]).transpose()
  bondcounts["Total"] = bondcounts["LL"] + bondcounts["RR"] + bondcounts["LR"]
  bondcounts = bondcounts.apply(lambda x : x / bondcounts["Total"])
  
  fig = go.Figure()
  fig.add_trace(go.Scatter(y=bondcounts["LL"],
                      mode='lines',
                      name='left homochiral'))
  fig.add_trace(go.Scatter(y=bondcounts["RR"],
                      mode='lines',
                      name='right homochiral'))
  fig.add_trace(go.Scatter(y=bondcounts["LR"],
                      mode='lines', name='heterochiral'))
  fig.update_layout(title='Proportion of LL, RR, and LR bonds by iteration',
                   xaxis_title='Iteration',
                   yaxis_title='Proportion')
  fig.update_yaxes(range=[0, 1])

  fig.show()

with tb.output_to("homochirality over time"):
  df = stats
  maxlen = df["Length"].max()

  fig = px.scatter(df, x="Length", y="%Homochirality",animation_frame="Iteration",
                   range_x=[-1,maxlen+1],range_y=[-0.01,1.2])
  fig.show()


with tb.output_to("homochirality vs length"):
  df = stats
  df = df.groupby("Length")['%Homochirality'].value_counts().reset_index(name='count')
  fig = px.scatter(df, x="Length",y="%Homochirality",size='count',range_y=(-0.1,1.1))
  fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  

  ###
  # Playing with graphing
  ###
  #fig.add_trace(go.Scatter(x=df["Length"],y=pars[0] + pars[1]/df["Length"]))


  # a = 5
  # b = 0.68

  # fig.add_trace(go.Scatter(x=df["Length"],y=a/df["Length"]+b))
  # fig.add_trace(go.Scatter(x=df["Length"],y=-a/df["Length"]+b))

  fig.show()



with tb.output_to("Homochirality vs age"):
  df = stats
  df = df.groupby("Age")['%Homochirality'].value_counts().reset_index(name='count')
  fig = px.scatter(df, x="Age",y="%Homochirality",size='count',trendline="lowess")
  fig.update_traces(marker=dict(line=dict(color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.show()





<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Ok! Go you. Nice job. Pat on the back. You're very successful.